In [3]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [78]:
%%time
import os
import ants
import numpy as np
from tqdm import tqdm

CPU times: user 0 ns, sys: 48 µs, total: 48 µs
Wall time: 81.5 µs


In [96]:
data_dir = '../Assets/fc_maps_orig/'
ofn_dir  = '../Assets/fc_maps_32smooth_new/'
ofn_dir_mats = ofn_dir.replace('fc_maps','fc_mats')

if not os.path.exists(ofn_dir):
    os.mkdir(ofn_dir)
    
if not os.path.exists(ofn_dir_mats):
    os.mkdir(ofn_dir_mats)

In [101]:
files = [file for file in os.listdir(data_dir) if file.endswith('.nii')]
files.sort()
n = len(files)

In [102]:
for s in tqdm(range(n)):
    
    if not os.path.exists(os.path.join(ofn_dir,files[s])):
        im = ants.image_read(os.path.join(data_dir,files[s]))
        if np.isnan(im.numpy()).sum()==0:
            #assert np.isnan(im.numpy()).sum()==0, 'NaNs in input - you fucked up'

            ## Processing Steps
            im = im+1
            im = im.smooth_image(sigma=3,sigma_in_physical_coordinates=False,FWHM=True,max_kernel_width=32)
            im = im.resample_image(resample_params=(32,32,32,51), use_voxels=True, interp_type=4)


            im[im>2]=2
            im[im<0]=0

        #     im3 = im.slice_image(3,1)
        #     im3.plot_ortho(flat=True,xyz_lines=False)
        #     print((im.numpy().min(),im.numpy().max()))

            assert np.isnan(im.numpy()).sum()==0, 'NaNs after processing - you fucked up'
            im.to_filename(os.path.join(ofn_dir,files[s]))
            np.save(os.path.join(ofn_dir_mats,files[s]),im.numpy())

100%|██████████| 1068/1068 [25:33<00:00,  1.44s/it]


In [ ]:
import shutil

In [135]:
files = [file for file in os.listdir(ofn_dir_mats) if file.endswith('.nii.npy')]
files.sort()
files[0:10]

[]

In [115]:
#.replace('.nii.npy','.npy')

In [134]:
for s in range(len(files)):
    infn = os.path.join(ofn_dir_mats,files[s])
    outfn = os.path.join(ofn_dir_mats,files[s]).replace('.nii.npy','.npy')
    shutil.move(src=infn, dst=outfn)